# GoogleNet

In [1]:
# https://github.com/dsai-asia/RTML/blob/main/Labs/02-PyTorch-AlexNet-GoogLeNet/02-PyTorch-AlexNet-GoogLeNet.ipynb
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
import time
import os
import copy
import torch.nn.functional as F

In [2]:
# from myModule import GoogLeNet
# model = GoogLeNet()
# model.eval()
# # alexnet = alexnet.to(device)

In [3]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train_dataset = torchvision.datasets.CIFAR10(root='../data', train=True, download=True, transform=preprocess)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=4, shuffle=True , num_workers=2)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print('Using device:', device)

from myModule import GoogLeNet

model = GoogLeNet()
model.eval()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
params_to_update = model.parameters()
optimizer = optim.SGD(params_to_update , lr=0.001, momentum=0.9)

Files already downloaded and verified
Using device: cuda:1


In [4]:
    
import numpy as np
# for inputs, labels in train_dataloader:
inputs = torch.tensor(np.ones([4, 3, 224, 224])).float()
inputs = inputs.to(device)

# print(inputs.shape)
# labels = torch.tensor(np.ones([4,1]))
# labels = labels.to(device)
outputs,aux4a,aux4d = model(inputs)
print(outputs.shape)
    # break

torch.Size([4, 10])
